In [2]:
import pandas as pd
import numpy as np
import requests
import geopandas, shapely
from pymongo import MongoClient,GEOSPHERE

In [3]:
def geo_frame(df):
    df = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df.latitude, df.longitude))
    df['geometry']=df['geometry'].apply(lambda x:shapely.geometry.mapping(x))
    return df

### Trying geo queries with differente collections

In [4]:
data = pd.read_csv("data\\test.csv")

In [5]:
data = data = geo_frame(data)

c:\users\juanp\anaconda3\envs\ironhack\lib\site-packages\geopandas\geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [6]:
# Set connection with MongoDB
conn = MongoClient("localhost:27017")
db = conn.get_database("final_project")
p = db.get_collection("places")
h = db.get_collection("homes")

In [28]:
def lugar(coords, categ, radio):
    return  list(p.find(
        {"category":categ,"geometry": {"$near": {
            "$geometry": {"type": "Point" ,
             "coordinates": coords
       }, "$maxDistance": radio}}},{"name":1, "longitude":1, "latitude":1, "category":1}
    ))
    

In [29]:
def sum_cat(coords):
    result = coords,len(lugar(coords, "gym"))
    if result[1] > 0:
        r = h.find_one({"latitude":result[0][0],"longitude":result[0][1]},{"_id":0,"province":0, "municipality":0, "showAdress":0, "geometry":0})
        return r
    else:
        pass

In [184]:
def lugar(coords, categ, radio):
    return list(p.find(
        {"category": categ, "geometry": {"$near": {
            "$geometry": {"type": "Point",
                          "coordinates": coords
                          }, "$maxDistance": radio}}}, {"name": 1, "longitude": 1, "latitude": 1, "category": 1}
    ))


def sum_gym(coords):
    result = coords, len(lugar(coords, "gym", 1000))
    if result[1] < 5:
        r = h.find_one({"latitude": result[0][0], "longitude": result[0][1]})
        return r
    else:
        pass


def sum_rest(coords):
    result = coords, len(lugar(coords, "restaurant", 5000))
    if result[1] > 15:
        r = h.find_one({"latitude": result[0][0], "longitude": result[0][1]})
        return r
    else:
        pass


def sum_night(coords):
    result = coords, len(lugar(coords, "nightlife", 5000))
    if result[1] > 15:
        r = h.find_one({"latitude": result[0][0], "longitude": result[0][1]})
        return r
    else:
        pass


def sum_superm(coords):
    result = coords, len(lugar(coords, "supermarket", 1500))
    if result[1] < 5:
        r = h.find_one({"latitude": result[0][0], "longitude": result[0][1]})
        return r
    else:
        pass


def sum_store(coords):
    result = coords, len(lugar(coords, "clothing_store", 2500))
    if result[1] > 10:
        r = h.find_one({"latitude": result[0][0], "longitude": result[0][1]})
        return r
    else:
        pass


def sum_medical(coords):
    result = coords, len(lugar(coords, "medical_centre", 3000))
    if result[1] < 5:
        r = h.find_one({"latitude": result[0][0], "longitude": result[0][1]})
        return r
    else:
        pass


def sum_transport(coords):
    result = coords, len(lugar(coords, "transport", 700))
    if result[1] < 5:
        r = h.find_one({"latitude": result[0][0], "longitude": result[0][1]})
        return r
    else:
        pass


def sum_ent(coords):
    result = coords, len(lugar(coords, "general_entertainment", 5000))
    if result[1] > 15:
        r = h.find_one({"latitude": result[0][0], "longitude": result[0][1]})
        return r
    else:
        pass


def sum_pharmacy(coords):
    result = coords, len(lugar(coords, "pharmacy", 500))
    if result[1] < 5:
        r = h.find_one({"latitude": result[0][0], "longitude": result[0][1]})
        return r
    else:
        pass


def sum_parks(coords):
    result = coords, len(lugar(coords, "park", 1500))
    if result[1] > 2:
        r = h.find_one({"latitude": result[0][0], "longitude": result[0][1]})
        return r
    else:
        pass

def sum_school(coords):
    result = coords, len(lugar(coords, "school", 5000))
    if result[1] > 5:
        r = h.find_one({"latitude": result[0][0], "longitude": result[0][1]})
        return r
    else:
        pass


In [212]:
def category(choice):

    if choice == "Supermarkets":
        return list(filter(None, map(sum_superm, coordinates)))

    elif choice == "Gyms":
        return list(filter(None, map(sum_gym, coordinates)))

    elif choice == "Clothing store":
        return list(filter(None, map(sum_store, coordinates)))

    elif choice == "Nightlife":
        return list(filter(None, map(sum_night, coordinates)))

    elif choice == "Transport":
        return list(filter(None, map(sum_transport, coordinates)))

    elif choice == "Entertainment":
        return list(filter(None, map(sum_ent, coordinates)))

    elif choice == "Parks":
        return list(filter(None, map(sum_parks, coordinates)))

    elif choice == "Restaurants":
        return list(filter(None, map(sum_rest, coordinates)))

    elif choice == "Pharmacy":
        return list(filter(None, map(sum_pharmacy, coordinates)))

    elif choice == "Hospital":
        return list(filter(None, map(sum_medical, coordinates)))

    elif choice == "School":
        return list(filter(None, map(sum_school, coordinates)))


def final(cat_1, cat_2, cat_3):
    lista = []

    lista.append(category(cat_1))
    lista.append(category(cat_2))
    lista.append(category(cat_3))

    x = pd.DataFrame(lista[0])
    y = pd.DataFrame(lista[1])
    z = pd.DataFrame(lista[2])
    
    result = pd.concat([x,y,z])  
    
    return result[result.groupby('latitude').latitude.transform('count')>1]

In [213]:
x = final("Hospital", "Entertainment", "Restaurant")

In [ ]:
df[df.Agent.groupby(df.Agent).transform('value_counts') > 1]


In [214]:
x

,_id,floor,price,propertyType,operation,size,exterior,rooms,bathrooms,address,...,district,neighborhood,latitude,longitude,showAddress,url,status,newDevelopment,hasLift,geometry
0,5fd49da7b73bfeea9c4b63c9,3,950.0,flat,rent,63.0,True,1,1,barrio Lavapiés-Embajadores,...,Centro,Lavapiés-Embajadores,40.405088,-3.699805,False,https://www.idealista.com/inmueble/87655984/,good,False,False,"{'type': 'Point', 'coordinates': [40.4050882, ..."
4,5fd49da7b73bfeea9c4b638b,1,1280.0,flat,rent,55.0,False,2,1,barrio Trafalgar,...,Chamberí,Trafalgar,40.434612,-3.698841,False,https://www.idealista.com/inmueble/87657202/,good,False,True,"{'type': 'Point', 'coordinates': [40.4346119, ..."
5,5fd49da7b73bfeea9c4b638c,2,2500.0,flat,rent,147.0,True,3,4,barrio Palacio,...,Centro,Palacio,40.413790,-3.707772,False,https://www.idealista.com/inmueble/87657168/,good,False,True,"{'type': 'Point', 'coordinates': [40.4137902, ..."
6,5fd49da7b73bfeea9c4b6393,1,1300.0,flat,rent,80.0,False,2,1,Calle de las Hileras,...,Centro,Sol,40.416493,-3.708165,False,https://www.idealista.com/inmueble/87657066/,good,False,False,"{'type': 'Point', 'coordinates': [40.416493, -..."
7,5fd49da7b73bfeea9c4b6395,3,1995.0,flat,rent,140.0,True,3,2,Lagasca,...,Salamanca,Castellana,40.432001,-3.683446,False,https://www.idealista.com/inmueble/87656962/,good,False,True,"{'type': 'Point', 'coordinates': [40.4320015, ..."
8,5fd49da7b73bfeea9c4b6396,5,799.0,flat,rent,35.0,False,1,1,Calle Rafael Herrera,...,Chamartín,Castilla,40.467131,-3.686489,False,https://www.idealista.com/inmueble/87656961/,renew,False,True,"{'type': 'Point', 'coordinates': [40.4671308, ..."
9,5fd49da7b73bfeea9c4b638b,1,1280.0,flat,rent,55.0,False,2,1,barrio Trafalgar,...,Chamberí,Trafalgar,40.434612,-3.698841,False,https://www.idealista.com/inmueble/87657202/,good,False,True,"{'type': 'Point', 'coordinates': [40.4346119, ..."
10,5fd49da7b73bfeea9c4b638c,2,2500.0,flat,rent,147.0,True,3,4,barrio Palacio,...,Centro,Palacio,40.413790,-3.707772,False,https://www.idealista.com/inmueble/87657168/,good,False,True,"{'type': 'Point', 'coordinates': [40.4137902, ..."
11,5fd49da7b73bfeea9c4b6393,1,1300.0,flat,rent,80.0,False,2,1,Calle de las Hileras,...,Centro,Sol,40.416493,-3.708165,False,https://www.idealista.com/inmueble/87657066/,good,False,False,"{'type': 'Point', 'coordinates': [40.416493, -..."
12,5fd49da7b73bfeea9c4b6395,3,1995.0,flat,rent,140.0,True,3,2,Lagasca,...,Salamanca,Castellana,40.432001,-3.683446,False,https://www.idealista.com/inmueble/87656962/,good,False,True,"{'type': 'Point', 'coordinates': [40.4320015, ..."


In [154]:
def final(cat_1, cat_2, cat_3):

    lista = []

    lista.append(category(cat_1))
    lista.append(category(cat_2))
    lista.append(category(cat_3))
    l = lista[0] + lista[1]
    
    return [dict(t) for t in {tuple(d.items()) for d in l}]

In [155]:
x = final("Transport", "Entertainment", "Restaurant")

In [158]:
type(x)

list

In [175]:
test = data.head(100)

In [193]:
test.to_csv("data\\prueba.csv")

In [176]:
coordinates = [list(i["coordinates"]) for i in test.geometry]

In [105]:
type(x)

str

In [110]:
x

['[]',
 "[{'floor': '4', 'price': 970.0, 'propertyType': 'flat', 'operation': 'rent', 'size': 58.0, 'exterior': False, 'rooms': 2, 'bathrooms': 1, 'address': 'Calle de Cuchilleros', 'district': 'Centro', 'neighborhood': 'Sol', 'latitude': 40.416321, 'longitude': -3.706578, 'showAddress': False, 'url': 'https://www.idealista.com/inmueble/87657327/', 'status': 'good', 'newDevelopment': False, 'hasLift': True}]",
 None]

In [67]:
lista = x[0]+x[1]

In [68]:
[dict(t) for t in {tuple(d.items()) for d in lista}]

AttributeError: 'str' object has no attribute 'items'